In [1]:
import numpy as np
import pandas as pd

class TransitionMatrices:
    def __init__(self):
        self.exploring = type('Exploring', (), {})()
        self.manipulating = type('Manipulating', (), {})()

        # Initialize all attributes with None
        for color in ['black', 'blue', 'red', 'nocolor']:
            setattr(self.exploring, color, None)
            setattr(self.manipulating, color, None)

def parse_transition_matrices(csv_file):
    """
    Reads the CSV and returns a TransitionMatrices object where you can access matrices like:
    transitions.exploring.blue
    transitions.manipulating.red
    etc.
    """
    transitions = TransitionMatrices()

    with open(csv_file, 'r') as f:
        lines = f.readlines()

    def extract_matrix(lines, start_idx, col_offset):
        """Helper function to extract a 2x2 matrix from the lines"""
        row1 = [float(x.replace(',', '.')) for x in lines[start_idx].split(',')[col_offset:col_offset+2] if x.strip()]
        row2 = [float(x.replace(',', '.')) for x in lines[start_idx+1].split(',')[col_offset:col_offset+2] if x.strip()]
        return np.array([row1, row2])

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        parts = line.split(',')

        # Check for color headers and matrices
        for color in ['black', 'blue', 'red', 'nocolor']:
            if color in line.lower() and len(parts[1].strip()) == len(color):
                # Skip the color line and the "To:" header line
                i += 2

                # Extract exploring matrix (columns 2-3)
                if i < len(lines) and 'From: No Behavior' in lines[i]:
                    exploring_matrix = extract_matrix(lines[i:], 0, 2)
                    setattr(transitions.exploring, color, exploring_matrix)

                # Extract manipulating matrix (columns 9-10)
                if i < len(lines) and 'From: No Behavior' in lines[i]:
                    manipulating_matrix = extract_matrix(lines[i:], 0, 9)
                    setattr(transitions.manipulating, color, manipulating_matrix)

        i += 1

    # Print all matrices for verification
    print("\nExploring Matrices:")
    for color in ['black', 'blue', 'red', 'nocolor']:
        matrix = getattr(transitions.exploring, color)
        if matrix is not None:
            print(f"\n{color}:")
            print(matrix)

    print("\nManipulating Matrices:")
    for color in ['black', 'blue', 'red', 'nocolor']:
        matrix = getattr(transitions.manipulating, color)
        if matrix is not None:
            print(f"\n{color}:")
            print(matrix)

    return transitions

def generate_binary_sequences(transition_matrix, length, n_sequences):
    """
    Generates multiple binary sequences based on a transition probability matrix.

    Args:
        transition_matrix: A 2x2 NumPy array of transition probabilities
        length: Length of each sequence
        n_sequences: Number of sequences to generate

    Returns:
        numpy.ndarray: Array of shape (n_sequences, length) containing the generated sequences
    """
    sequences = np.zeros((n_sequences, length), dtype=int)

    for seq_idx in range(n_sequences):
        current_state = 0  # Start in state 0
        for i in range(1, length):
            if current_state == 0:
                if np.random.rand() < transition_matrix[0, 1]:  # Transition to 1
                    current_state = 1
            else:  # current_state == 1
                if np.random.rand() < transition_matrix[1, 0]:  # Transition to 0
                    current_state = 0
            sequences[seq_idx, i] = current_state

    return sequences

if __name__ == "__main__":
    transitions = parse_transition_matrices('/root/rabita_transitions.csv')

    # Example: generate sequences for exploring blue
    print("\nExample: Generating sequences using exploring.blue matrix:")
    matrix = transitions.exploring.blue
    sequences = generate_binary_sequences(matrix, length=100, n_sequences=5)
    print("Matrix used:")
    print(matrix)
    print("\nSequences shape:", sequences.shape)
    print("First sequence:")
    print(sequences[0])


Exploring Matrices:

black:
[[0.997 0.003]
 [0.295 0.705]]

blue:
[[0.994 0.006]
 [0.425 0.575]]

red:
[[0.996 0.004]
 [0.418 0.582]]

nocolor:
[[0.997 0.003]
 [0.317 0.683]]

Manipulating Matrices:

black:
[[0.998 0.002]
 [0.178 0.822]]

blue:
[[0.996 0.004]
 [0.128 0.872]]

red:
[[0.995 0.005]
 [0.063 0.937]]

nocolor:
[[0.997 0.003]
 [0.213 0.787]]

Example: Generating sequences using exploring.blue matrix:
Matrix used:
[[0.994 0.006]
 [0.425 0.575]]

Sequences shape: (5, 100)
First sequence:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
